In [58]:
import utils
import glob
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from hydroeval import evaluator, nse, rmse, pbias
pd.options.mode.chained_assignment = None

In [74]:
sims = [12, 57, 75]
obds = ['str_012', 'str_057','str_075']
stdate = '1/1/1980'
time_step ='M'
caldate = '1/1/1992'
eddate = '12/31/2001'

In [75]:
df = utils.get_sim_obd('Animas', stdate, time_step, sims, obds, caldate, eddate)
df

,sim_012,sim_057,sim_075,str_012,str_057,str_075
1992-01-31,0.8408,11.630,11.780,1.14,8.10,8.24
1992-02-29,1.0360,10.410,10.830,1.30,7.98,9.07
1992-03-31,0.9574,9.539,12.430,1.69,12.86,12.74
1992-04-30,9.0710,55.630,56.290,5.10,40.43,30.79
1992-05-31,15.1600,38.070,38.690,18.74,87.18,71.35
...,...,...,...,...,...,...
2001-08-31,14.8900,39.860,40.140,6.59,19.71,15.15
2001-09-30,9.6740,15.840,15.940,2.76,9.16,4.12
2001-10-31,2.0960,9.534,9.646,2.15,7.61,4.69
2001-11-30,0.4105,5.657,5.754,1.33,7.45,6.64


In [83]:
def get_stats(df):
    df_stat = df.dropna()

    sim = df_stat.iloc[:, 0].to_numpy()
    obd = df_stat.iloc[:, 1].to_numpy()
    df_nse = evaluator(nse, sim, obd)[0]
    df_rmse = evaluator(rmse, sim, obd)[0]
    df_pibas = evaluator(pbias, sim, obd)[0]
    r_squared = (
        ((sum((obd - obd.mean())*(sim-sim.mean())))**2)/
        ((sum((obd - obd.mean())**2)* (sum((sim-sim.mean())**2))))
        )
    return df_nse, df_rmse, df_pibas, r_squared

In [90]:
stat_df = pd.DataFrame()
for i in range(len(sims)):
    exdf = df.iloc[:, [i, i+len(sims)]]
    df_list = get_stats(exdf)
    stat_series = pd.Series(df_list, name='Reach {}'.format(sims[i]))
    stat_df = pd.concat([stat_df, stat_series], axis=1)
    # stat_df.reindex(['NSE', 'RMSE', 'PBIAS', 'R Squared'])

In [92]:
stat_df.index =['NSE', 'RMSE', 'PBIAS', 'R Squared']

In [93]:
stat_df

,Reach 12,Reach 57,Reach 75
NSE,0.551042,0.672355,0.597816
RMSE,6.913082,17.294399,18.402982
PBIAS,8.770359,-1.548486,-12.262726
R Squared,0.573608,0.719828,0.687675


In [23]:
df_list.tolist()

AttributeError: 'tuple' object has no attribute 'tolist'

In [9]:
type(df_list)

tuple

In [104]:
fig = go.Figure()
colors = (get_matplotlib_cmap('tab10', bins=8))

for i in range(len(sims)):
    fig.add_trace(go.Scatter(
        x=ani_df.index, y=ani_df.iloc[:, i], name='Reach {}'.format(sims[i]),
        line=dict(color=colors[i], width=2),
        legendgroup='Reach {}'.format(sims[i])
        ))
for i in range(len(sims)):
    fig.add_trace(go.Scatter(
        x=ani_df.index, y=ani_df.iloc[:, i+4], mode='markers', name='Observed {}'.format(sims[i]),
        marker=dict(color=colors[i]),
        legendgroup='Reach {}'.format(sims[i]),

        showlegend=False
        ))
# line_fig = px.line(ani_df, height=500, width=1200)
fig.update_layout(
    # showlegend=False,
    plot_bgcolor='white',
    height=500,
    width=1200
)
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgray', title='')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray', title='Stream Discharge (m<sup>3</sup>/s)')

fig.update_layout(legend=dict(
    yanchor="top",
    y=1.2,
    xanchor="center",
    x=0.5,
    orientation="h",
    title='',
))
fig.update_traces(marker=dict(size=10, opacity=0.5,
                              line=dict(width=1,
                                        color='white')
                                        ),
                  selector=dict(mode='markers'))
                  


In [46]:
line_fig = px.line(ani_df, height=500, width=1200)
line_fig.update_layout(
    # showlegend=False,
    plot_bgcolor='white',
)
line_fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgray', title='')
line_fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray', title='Stream Discharge (m<sup>3</sup>/s)')

line_fig.update_layout(legend=dict(
    yanchor="top",
    y=1.2,
    xanchor="center",
    x=0.5,
    orientation="h",
    title=''
))

In [90]:
import itertools
from matplotlib import cm
import numpy as np

In [101]:
def get_matplotlib_cmap(cmap_name, bins, alpha=1):
    if bins is None:
        bins = 10
    cmap = cm.get_cmap(cmap_name)
    h = 1.0 / bins
    contour_colour_list = []

    for k in range(bins):
        C = list(map(np.uint8, np.array(cmap(k * h)[:3]) * 255))
        contour_colour_list.append('rgba' + str((C[0], C[1], C[2], alpha)))

    C = list(map(np.uint8, np.array(cmap(bins * h)[:3]) * 255))
    contour_colour_list.append('rgba' + str((C[0], C[1], C[2], alpha)))
    return contour_colour_list


In [92]:
PALETTE = itertools.cycle(get_matplotlib_cmap('tab10', bins=8))

In [93]:
PALETTE

In [102]:
hi = (get_matplotlib_cmap('tab10', bins=8))

In [103]:
hi

['rgba(31, 119, 180, 1)',
 'rgba(255, 127, 14, 1)',
 'rgba(44, 160, 44, 1)',
 'rgba(214, 39, 40, 1)',
 'rgba(140, 86, 75, 1)',
 'rgba(227, 119, 194, 1)',
 'rgba(127, 127, 127, 1)',
 'rgba(188, 189, 34, 1)',
 'rgba(23, 190, 207, 1)']